In [2]:
import os
import pandas as pd
import re
import holoviews as hv
import ipywidgets as widgets
from IPython.display import display
import ninetysix as ns
import numpy as np
hv.extension('bokeh')

In [3]:
def work_up_lcms(
    file,
    products,
    substrates=None,
    drop_string=None,
):
    """Works up a standard csv file from Revali.
    Parameters:
    -----------
    file: string
        Path to the csv file
    products: list of strings
        Name of the peaks that correspond to the product
    substrates: list of strings
        Name of the peaks that correspond to the substrate
    drop_string: string, default 'burn_in'
        Name of the wells to drop, e.g., for the wash/burn-in period that are not samples.
    Returns:
    --------
    plate: ns.Plate object (DataFrame-like)
    """
    # Read in the data
    df = pd.read_csv(file, header=[1])
    # Convert nans to 0
    df = df.fillna(0)
    # Only grab the Sample Acq Order No.s that have a numeric value
    index = [True for _ in df['Sample Acq Order No']]
    for i, value in enumerate(df['Sample Acq Order No']):
        try:
            int(value)
        except ValueError:
            index[i] = False
    # Index on this
    df = df[index]
    def fill_vial_number(series):
        for i, row in enumerate(series):
            if pd.isna(row):
                series[i] = series[i-1]
        return series
    df['Sample Vial Number'] = fill_vial_number(df['Sample Vial Number'].copy())
    # Remove unwanted wells
    df = df[df['Sample Name'] != drop_string]
    # Get wells
    df.insert(0, 'Well', df['Sample Vial Number'].apply(lambda x: x.split('-')[-1]))
    # Rename
    df = df.rename({'Sample Name': 'Plate'}, axis='columns')
    # Create minimal DataFrame
    df = df[['Well', 'Plate', 'Compound Name', 'Area']].reset_index(drop=True)
    # Pivot table; drop redundant values by only taking 'max' with aggfunc
    # (i.e., a row is (value, NaN, NaN) and df is 1728 rows long;
    # taking max to aggregate duplicates gives only (value) and 576 rows long)
    df = df.pivot_table(
        index=['Well', 'Plate'],
        columns='Compound Name',
        values='Area',
        aggfunc='max'
    ).reset_index()
    # Get rows and columns
    df.insert(1, 'Column', df['Well'].apply(lambda x: int(x[1:])))
    df.insert(1, 'Row', df['Well'].apply(lambda x: x[0]))
    # Set values as floats
    cols = products+substrates if substrates is not None else products
    for col in cols:
        df[col] = df[col].astype(float)
    plate = ns.Plate(df, value_name=products[-1]).set_as_location('Plate', idx=3)
    plate.values = products
    return plate

In [7]:
# Function to process the plate files
def process_plate_files(product):
    # Load the provided CSV file
    results_df = pd.read_csv('HMC0225_HMC0226.csv')

    # Extract the required columns: Plate, Well, Mutations, and nc_variant, and remove rows with '#N.A.#' and NaN values
    filtered_df = results_df[['Plate', 'Well', 'Mutations', 'nc_variant', 'aa_variant']]
    filtered_df = filtered_df[(filtered_df['Mutations'] != '#N.A.#')].dropna()

    # Extract the unique entries of Plate
    unique_plates = filtered_df['Plate'].unique()

    # Create an empty list to store the processed plate data
    processed_data = []

    # Iterate over unique Plates and search for corresponding CSV files in the current directory
    for plate in unique_plates:
        # Construct the expected filename based on the Plate value
        filename = f"{plate}.csv"
        
        # Check if the file exists in the current directory
        if os.path.isfile(filename):
            # Work up data to plate object
            plate_object = work_up_lcms(filename, product)
            
            # Extract attributes from plate_object as needed for downstream processes
            if hasattr(plate_object, 'df'):
                # Assuming plate_object has a dataframe-like attribute 'df' that we can work with
                plate_df = plate_object.df
                plate_df['Plate'] = plate  # Add the plate identifier for reference
                
                # Merge filtered_df with plate_df to retain Mutations and nc_variant columns
                merged_df = pd.merge(plate_df, filtered_df, on=['Plate', 'Well'], how='left')
                columns_order = ['Plate', 'Well', 'Row', 'Column', 'Mutations'] + product + ['nc_variant', 'aa_variant']
                merged_df = merged_df[columns_order]
                processed_data.append(merged_df)

    # Concatenate all dataframes if available
    if processed_data:
        processed_df = pd.concat(processed_data, ignore_index=True)
    else:
        processed_df = pd.DataFrame(columns=['Plate', 'Well', 'Row', 'Column', 'Mutations'] + product + ['nc_variant', 'aa_variant'])

    # Ensure all entries in 'Mutations' are treated as strings
    processed_df['Mutations'] = processed_df['Mutations'].astype(str)

    # Remove any rows with empty values
    processed_df = processed_df.dropna()

    # Optionally, save the processed DataFrame to a CSV file
    processed_df.to_csv('processed_plate_data.csv', index=False)

    # Create a subset of the processed DataFrame where only '#PARENT#' and variants are kept
    # parent_and_variant_df = processed_df[processed_df['Mutations'].isin(['#PARENT#']) | (processed_df['Mutations'] != '#N.A.#')]
    # parent_and_variant_df.to_csv('/group_files/levseq-sequence-function/HMC0225_HMC0226/Results/parent_and_variant_data.csv', index=False)

    # Return the processed DataFrame for downstream processes
    return processed_df

# Function to identify SSM experiments
def identify_ssm_experiments(df):
    # Function to identify if a mutation is part of an SSM experiment
    def is_ssm(mutation):
        # Define a regex pattern for detecting SSM: a single position with varying amino acids
        return bool(re.match(r"^[A-Z]\d+[A-Z]$", str(mutation)))

    # Add a new column indicating whether each mutation is part of an SSM experiment
    df['is_ssm'] = df['Mutations'].apply(is_ssm)
    return df

# Example usage of the function
processed_plate_df = process_plate_files(product=["pdt"])
#processed_plate_df = identify_ssm_experiments(processed_plate_df)
processed_plate_df